In [98]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV

import os
print(os.listdir(r"D:\Dropbox\Wisconsin School of Business\MSBA\CS 532\Jupyter NB\Final Project\input"))

['test.csv', 'train.csv']


In [99]:
train = shuffle(pd.read_csv(r"D:\Dropbox\Wisconsin School of Business\MSBA\CS 532\Jupyter NB\Final Project\input\train.csv"))
print(train.shape)
test = shuffle(pd.read_csv(r"D:\Dropbox\Wisconsin School of Business\MSBA\CS 532\Jupyter NB\Final Project\input\test.csv"))
print(test.shape)

(7352, 563)
(2947, 563)


In [3]:

train_outcome = pd.crosstab(index=train["Activity"],  # Make a crosstab
                              columns="count")      # Name the count column

train_outcome

col_0,count
Activity,
LAYING,1407
SITTING,1286
STANDING,1374
WALKING,1226
WALKING_DOWNSTAIRS,986
WALKING_UPSTAIRS,1073


In [100]:
#Create the binary class variables for train set:
laying = []
for i in range(len(train)):
    if train['Activity'][i] == "LAYING":
        laying.append(1)
    else:
        laying.append(0)
#train['LAYING'] = laying
train_laying = np.c_[laying]

sitting = []
for i in range(len(train)):
    if train['Activity'][i] == "SITTING":
        sitting.append(1)
    else:
        sitting.append(0)
#train['SITTING'] = sitting
train_sitting = np.c_[sitting]

standing = []
for i in range(len(train)):
    if train['Activity'][i] == "STANDING":
        standing.append(1)
    else:
        standing.append(0)
#train['STANDING'] = standing
train_standing = np.c_[standing]

walking = []
for i in range(len(train)):
    if train['Activity'][i] == "WALKING":
        walking.append(1)
    else:
        walking.append(0)
#train['WALKING'] = walking
train_walking = np.c_[walking]

walkingdownstairs = []
for i in range(len(train)):
    if train['Activity'][i] == "WALKING_DOWNSTAIRS":
        walkingdownstairs.append(1)
    else:
        walkingdownstairs.append(0)
#train['WALKING_DOWNSTAIRS'] = walkingdownstairs
train_walkingdownstairs = np.c_[walkingdownstairs]

walkingupstairs = []
for i in range(len(train)):
    if train['Activity'][i] == "WALKING_UPSTAIRS":
        walkingupstairs.append(1)
    else:
        walkingupstairs.append(0)
#train['WALKING_UPSTAIRS'] = walkingupstairs
train_walkingupstairs = np.c_[walkingupstairs]

#Create the binary class variables for test set:
test_laying = []
for i in range(len(test)):
    if test['Activity'][i] == "LAYING":
        test_laying.append(1)
    else:
        test_laying.append(0)
#test['LAYING'] = laying
test_laying = np.c_[test_laying]

test_sitting = []
for i in range(len(test)):
    if test['Activity'][i] == "SITTING":
        test_sitting.append(1)
    else:
        test_sitting.append(0)
#test['SITTING'] = sitting
test_sitting = np.c_[test_sitting]

test_standing = []
for i in range(len(test)):
    if test['Activity'][i] == "STANDING":
        test_standing.append(1)
    else:
        test_standing.append(0)
#test['STANDING'] = standing
test_standing = np.c_[test_standing]

test_walking = []
for i in range(len(test)):
    if test['Activity'][i] == "WALKING":
        test_walking.append(1)
    else:
        test_walking.append(0)
#test['WALKING'] = walking
test_walking = np.c_[test_walking]

test_walkingdownstairs = []
for i in range(len(test)):
    if test['Activity'][i] == "WALKING_DOWNSTAIRS":
        test_walkingdownstairs.append(1)
    else:
        test_walkingdownstairs.append(0)
#test['WALKING_DOWNSTAIRS'] = walkingdownstairs
test_walkingdownstairs = np.c_[test_walkingdownstairs]

test_walkingupstairs = []
for i in range(len(test)):
    if test['Activity'][i] == "WALKING_UPSTAIRS":
        test_walkingupstairs.append(1)
    else:
        test_walkingupstairs.append(0)
#test['WALKING_UPSTAIRS'] = walkingupstairs
test_walkingupstairs = np.c_[test_walkingupstairs]


In [102]:
# Seperating Predictors and Outcome values from train and test sets
X_train = pd.DataFrame(train.drop(['Activity','subject'],axis=1))
#Y_train_label = train.Activity.values.astype(object)
X_test = pd.DataFrame(test.drop(['Activity','subject'],axis=1))
#Y_test_label = test.Activity.values.astype(object)

# Dimension of Train and Test set 
print("Dimension of Train set",X_train.shape)
print("Dimension of Test set",X_test.shape,"\n")


Dimension of Train set (7352, 561)
Dimension of Test set (2947, 561) 



In [187]:
U,s,VT = np.linalg.svd(X_train,full_matrices=False)

lamb = 0.00000002

d=[]
for k in range(len(s)):
    d.append(s[k] / (s[k]**2 + lamb))

S = np.diag(d)
print(S)

[[7.20405334e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.54231100e-03 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 7.42681303e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 8.03952356e-08
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  5.36423942e-08 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.79041945e-08]]


In [188]:
#Ridge Regression for six different models
w_laying = VT.transpose()@S@U.transpose()@train_laying
w_sitting = VT.transpose()@S@U.transpose()@train_sitting
w_standing = VT.transpose()@S@U.transpose()@train_standing
w_walking = VT.transpose()@S@U.transpose()@train_walking
w_walkingdownstairs = VT.transpose()@S@U.transpose()@train_walkingdownstairs
w_walkingupstairs = VT.transpose()@S@U.transpose()@train_walkingupstairs


(561, 561)
(561, 561)
(7352, 561)
(7352, 1)


In [190]:
test_laying_pred = np.sign(X_test@w_laying)
test_sitting_pred = np.sign(X_test@w_sitting)
test_standing_pred = np.sign(X_test@w_standing)
test_walking_pred = np.sign(X_test@w_walking)
test_walkingdownstairs_pred = np.sign(X_test@w_walkingdownstairs)
test_walkingupstairs_pred = np.sign(X_test@w_walkingupstairs)


error_vec_laying = [0 if j[0]==j[1] else 1 for j in np.hstack((test_laying_pred, test_laying))]
print(sum(error_vec_laying)/len(test))

error_vec_sitting = [0 if j[0]==j[1] else 1 for j in np.hstack((test_sitting_pred, test_sitting))]
print(sum(error_vec_sitting)/len(test))

error_vec_standing = [0 if j[0]==j[1] else 1 for j in np.hstack((test_standing_pred, test_standing))]
print(sum(error_vec_standing)/len(test))

error_vec_walking = [0 if j[0]==j[1] else 1 for j in np.hstack((test_walking_pred, test_walking))]
print(sum(error_vec_walking)/len(test))

error_vec_walkingdownstairs = [0 if j[0]==j[1] else 1 for j in np.hstack((test_walkingdownstairs_pred, test_walkingdownstairs))]
print(sum(error_vec_walkingdownstairs)/len(test))

error_vec_walkingupstairs = [0 if j[0]==j[1] else 1 for j in np.hstack((test_walkingupstairs_pred, test_walkingupstairs))]
print(sum(error_vec_walkingupstairs)/len(test))

# separately, our models do not do very well in predicting

0.8242280285035629
0.841873091279267
0.831014591109603
0.8411944350186631
0.8713946386155412
0.8537495758398371


In [220]:
test_laying_pred = X_test@w_laying
test_sitting_pred = X_test@w_sitting
test_standing_pred = X_test@w_standing
test_walking_pred = X_test@w_walking
test_walkingdownstairs_pred = X_test@w_walkingdownstairs
test_walkingupstairs_pred = X_test@w_walkingupstairs

pred = np.hstack((test_laying_pred, test_sitting_pred, test_standing_pred, test_walking_pred, test_walkingdownstairs_pred, test_walkingupstairs_pred))

pred

array([[ 0.23487388,  0.12769207,  0.22979056,  0.25649339,  0.14635855,
         0.00479156],
       [ 0.14869078, -0.06562864,  0.1495836 ,  0.09715062,  0.25152887,
         0.41867477],
       [ 0.22399248,  0.22898312,  0.29020362,  0.10863769,  0.07757342,
         0.07060968],
       ...,
       [ 0.19968976,  0.16750997,  0.20261033,  0.16076832,  0.02200941,
         0.24741221],
       [ 0.22923436,  0.1800778 ,  0.20448946,  0.22790128,  0.22059572,
        -0.06229861],
       [ 0.22169074,  0.11272059,  0.31114482,  0.13987983,  0.26957409,
        -0.05501006]])

In [222]:
# Choose the classifier with the highest confidence score
for i in range(len(pred)):
    ab = np.abs(pred[i])
    index = np.argmax(ab)
    for j in range(6):
        if j != index:
            pred[i][j] = 0

array([[0.        , 0.        , 0.        , 0.25649339, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.41867477],
       [0.        , 0.        , 0.29020362, 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.24741221],
       [0.22923436, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.31114482, 0.        , 0.        ,
        0.        ]])

In [233]:
#reassign our prediction
test_laying_pred = np.sign(pred[:,0])
test_sitting_pred = np.sign(pred[:,1])
test_standing_pred = np.sign(pred[:,2])
test_walking_pred = np.sign(pred[:,3])
test_walkingdownstairs_pred = np.sign(pred[:,4])
test_walkingupstairs_pred = np.sign(pred[:,5])

In [234]:
#final error rate
error_vec_laying = [0 if j[0]==j[1] else 1 for j in np.hstack((np.c_[test_laying_pred], test_laying))]
print(sum(error_vec_laying)/len(test))

error_vec_sitting = [0 if j[0]==j[1] else 1 for j in np.hstack((np.c_[test_sitting_pred], test_sitting))]
print(sum(error_vec_sitting)/len(test))

error_vec_standing = [0 if j[0]==j[1] else 1 for j in np.hstack((np.c_[test_standing_pred], test_standing))]
print(sum(error_vec_standing)/len(test))

error_vec_walking = [0 if j[0]==j[1] else 1 for j in np.hstack((np.c_[test_walking_pred], test_walking))]
print(sum(error_vec_walking)/len(test))

error_vec_walkingdownstairs = [0 if j[0]==j[1] else 1 for j in np.hstack((np.c_[test_walkingdownstairs_pred], test_walkingdownstairs))]
print(sum(error_vec_walkingdownstairs)/len(test))

error_vec_walkingupstairs = [0 if j[0]==j[1] else 1 for j in np.hstack((np.c_[test_walkingupstairs_pred], test_walkingupstairs))]
print(sum(error_vec_walkingupstairs)/len(test))

0.31150322361723787
0.33050559891415
0.33389888021717
0.2578893790295215
0.18595181540549713
0.2544960977265015
